In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable

#### 이전에는 one_hot으로 인코딩된 input을 썼는데 이번에는 lookup table을 활용해보자

In [9]:
idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

In [10]:
# 인덱스만 LongTensor로 저장한다. 
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

In [11]:
num_classes = 5      # h, i, e, l, o 이렇게 5개
input_size = 5  
embedding_size = 10  # embedding size구
hidden_size = 5      # output from the LSTM. 5 to directly predict one-hot
batch_size = 1       # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1       # one-layer rnn

In [12]:
class Model(nn.Module):
    def __init__(self,num_classes, input_size, hidden_size, num_layers):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(num_classes, embedding_size) # Vocab_size, Vector_size
        
        self.num_classes = num_classes          # h,i,e,l,o 이렇게 5개
        self.num_layers = num_layers            # one-layer-rnn
        self.input_size = input_size            # input data size
        self.hidden_size = hidden_size          # hidden layer size
        self.sequence_length = sequence_length  # i h e l l o 6개
        
        self.rnn = nn.RNN(input_size = embedding_size,
                          hidden_size = 5, batch_first = True)
                                                 # batch first 해줘야 input data가 (batch_size, seq_length, input_size)
        
        self.fc = nn.Linear(hidden_size, num_classes)
                                                 # 왜 Fully_connected를 넣어준거지?
        
    def forward(self,x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)) # 초기값 0으로 지정
        
        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1) # 앞에서 input이랑 같은 모습으로 바꿔줌
        
        out, _ = self.rnn(emb,h_0)                      # out은 6X5 (batch, sequence_length, input_size)가 된다.
        return self.fc(out.view(-1, num_classes))       
        

In [13]:
# Instantiate RNN model
model = Model(num_classes, input_size, hidden_size, num_layers)
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [14]:
# Train the model
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

epoch: 1, loss: 1.633
Predicted string:  lhlell
epoch: 2, loss: 1.367
Predicted string:  lillll
epoch: 3, loss: 1.162
Predicted string:  ihelll
epoch: 4, loss: 0.960
Predicted string:  ihelll
epoch: 5, loss: 0.793
Predicted string:  ihelll
epoch: 6, loss: 0.665
Predicted string:  ehelll
epoch: 7, loss: 0.561
Predicted string:  ihello
epoch: 8, loss: 0.460
Predicted string:  ihello
epoch: 9, loss: 0.377
Predicted string:  ihello
epoch: 10, loss: 0.298
Predicted string:  ihello
epoch: 11, loss: 0.236
Predicted string:  ihello
epoch: 12, loss: 0.186
Predicted string:  ihello
epoch: 13, loss: 0.147
Predicted string:  ihello
epoch: 14, loss: 0.117
Predicted string:  ihello
epoch: 15, loss: 0.094
Predicted string:  ihello
epoch: 16, loss: 0.076
Predicted string:  ihello
epoch: 17, loss: 0.062
Predicted string:  ihello
epoch: 18, loss: 0.050
Predicted string:  ihello
epoch: 19, loss: 0.041
Predicted string:  ihello
epoch: 20, loss: 0.034
Predicted string:  ihello
epoch: 21, loss: 0.029
Predic